In [1]:
import datetime
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras

from model import build_encoder, build_autoencoder
from data import example_to_tensor, normalize, add_channel_axis, train_test_split
from utils import duplicate_iterator, plot_slice, plot_animated_volume
from config import allocate_gpu_memory_only_when_needed, data_root_dir, seed

%matplotlib inline
plt.rcParams["figure.figsize"] = [15, 7]

In [2]:
# from tensorflow.keras.mixed_precision import experimental as mixed_precision
#
# policy = mixed_precision.Policy("mixed_float16")
##policy = mixed_precision.Policy("float32")
# mixed_precision.set_policy(policy)
# print("Compute dtype: %s" % policy.compute_dtype)
# print("Variable dtype: %s" % policy.variable_dtype)

In [3]:
downscaling = 2
if downscaling == 4:
    input_shape = (24, 128, 128, 1)
    neg_tfrecord_glob = "covid-neg-0.25/*.tfrecord"
    pos_tfrecord_glob = "covid-pos-0.25/*.tfrecord"
elif downscaling == 2:
    input_shape = (48, 256, 256, 1)
    neg_tfrecord_glob = "covid-neg-0.25/*.tfrecord"
    pos_tfrecord_glob = "covid-pos-0.25/*.tfrecord"
elif downscaling == 1:
    input_shape = (96, 512, 512, 1)
    neg_tfrecord_glob = "covid-neg-0.25/*.tfrecord"
    pos_tfrecord_glob = "covid-pos-0.25/*.tfrecord"
else:
    raise RuntimeError("Downscaling not supported")

encoder_num_filters = [32, 64, 128]
epochs = 1000
patience = 20
batch_size = 2
learning_rate = 0.00001  # very low learning rate for transfer learning
val_perc = 0.1  # percentage from the already splitted training test
test_perc = 0.1

In [4]:
neg_tfrecord_fnames = [str(p) for p in Path(data_root_dir).glob(neg_tfrecord_glob)]
neg_x = (
    tf.data.TFRecordDataset(neg_tfrecord_fnames)
    .map(example_to_tensor, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .map(normalize, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .map(add_channel_axis, num_parallel_calls=tf.data.experimental.AUTOTUNE)
)
num_neg = sum(1 for _ in neg_x)
print(f"Number of negative samples: {num_neg}")
neg_x

Number of negative samples: 30


<ParallelMapDataset shapes: (None, None, None, 1), types: tf.float32>

In [5]:
pos_tfrecord_fnames = [str(p) for p in Path(data_root_dir).glob(pos_tfrecord_glob)]
pos_x = (
    tf.data.TFRecordDataset(pos_tfrecord_fnames)
    .map(example_to_tensor, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .map(normalize, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .map(add_channel_axis, num_parallel_calls=tf.data.experimental.AUTOTUNE)
)
num_pos = sum(1 for _ in pos_x)
print(f"Number of positive samples: {num_pos}")
pos_x

Number of positive samples: 30


<ParallelMapDataset shapes: (None, None, None, 1), types: tf.float32>

In [6]:
neg_y = tf.data.Dataset.from_tensors(tf.constant([0], dtype=tf.int8)).repeat(num_neg)
neg_dataset = tf.data.Dataset.zip((neg_x, neg_y))
neg_dataset

<ZipDataset shapes: ((None, None, None, 1), (1,)), types: (tf.float32, tf.int8)>

In [7]:
pos_y = tf.data.Dataset.from_tensors(tf.constant([1], dtype=tf.int8)).repeat(num_pos)
pos_dataset = tf.data.Dataset.zip((pos_x, pos_y))
pos_dataset

<ZipDataset shapes: ((None, None, None, 1), (1,)), types: (tf.float32, tf.int8)>

In [8]:
dataset = neg_dataset.concatenate(pos_dataset)
dataset, test_dataset = train_test_split(
    dataset,
    test_perc=test_perc,
    cardinality=None,
    seed=seed,
)
test_dataset = test_dataset.padded_batch(1, (input_shape, (1,)))
train_dataset, val_dataset = train_test_split(
    dataset,
    test_perc=val_perc,
    cardinality=None,
    seed=seed,
)
val_dataset = (
    val_dataset.padded_batch(batch_size, (input_shape, (1,)), drop_remainder=True)
    .cache()
    .prefetch(tf.data.experimental.AUTOTUNE)
)
train_dataset = (
    train_dataset.padded_batch(batch_size, (input_shape, (1,)), drop_remainder=True)
    .cache()  # must be called before shuffle
    .shuffle(buffer_size=64, reshuffle_each_iteration=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)
train_dataset

<PrefetchDataset shapes: ((2, 48, 256, 256, 1), (2, 1)), types: (tf.float32, tf.int8)>

In [9]:
from collections import Counter

print(Counter(label.numpy()[0] for _, label in train_dataset.unbatch()))
print(Counter(label.numpy()[0] for _, label in val_dataset.unbatch()))
print(Counter(label.numpy()[0] for _, label in test_dataset.unbatch()))

Counter({0: 26, 1: 22})
Counter({1: 2, 0: 2})
Counter({1: 5, 0: 1})


In [10]:
pretrained = False
if pretrained:
    encoder = keras.models.load_model(
        "models/autoencoder-20201023-112618.h5"
    ).get_layer("encoder")
    encoder.trainable = False  # free the pre-trained encoder
else:
    encoder = build_encoder(input_shape)
encoder.summary()

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 256, 256, 1)] 0         
_________________________________________________________________
conv3d (Conv3D)              (None, 48, 256, 256, 64)  1792      
_________________________________________________________________
alpha_dropout (AlphaDropout) (None, 48, 256, 256, 64)  0         
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 24, 128, 128, 64)  0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 24, 128, 128, 128) 221312    
_________________________________________________________________
alpha_dropout_1 (AlphaDropou (None, 24, 128, 128, 128) 0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 12, 64, 64, 128)   0   

In [11]:
cnn = keras.Sequential(
    [
        encoder,
        keras.layers.Flatten(),
        keras.layers.Dense(
            512,
            kernel_initializer="lecun_normal",
            bias_initializer="lecun_normal",
            activation="selu",
        ),
        keras.layers.AlphaDropout(0.3),
        keras.layers.Dense(1, activation="sigmoid"),
    ],
    name="cnn",
)
cnn.summary()

Model: "cnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder (Functional)         (None, 6, 32, 32, 256)    1108096   
_________________________________________________________________
flatten (Flatten)            (None, 1572864)           0         
_________________________________________________________________
dense (Dense)                (None, 512)               805306880 
_________________________________________________________________
alpha_dropout_3 (AlphaDropou (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 513       
Total params: 806,415,489
Trainable params: 806,415,489
Non-trainable params: 0
_________________________________________________________________


In [ ]:
cnn.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=[
        # keras.metrics.TruePositives(name="tp"),
        # keras.metrics.FalsePositives(name="fp"),
        # keras.metrics.TrueNegatives(name="tn"),
        # keras.metrics.FalseNegatives(name="fn"),
        keras.metrics.BinaryAccuracy(name="accuracy"),
        # keras.metrics.Precision(name="precision"),
        # keras.metrics.Recall(name="recall"),
        # keras.metrics.AUC(name="auc"),
    ],
)
monitor_metric = "val_accuracy"

start_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
best_checkpoint = f"models/{'pretrained-' if pretrained else ''}{start_time}.h5"
checkpoint_cb = keras.callbacks.ModelCheckpoint(
    best_checkpoint, monitor=monitor_metric, mode="max", verbose=1, save_best_only=True
)
early_stopping_cb = keras.callbacks.EarlyStopping(
    monitor=monitor_metric, patience=patience, mode="max"
)
log_dir = f"logs/{'pretrained-' if pretrained else ''}{start_time}"
file_writer = tf.summary.create_file_writer(log_dir)
tensorboard_cb = tf.keras.callbacks.TensorBoard(
    log_dir=f"logs/{'pretrained-' if pretrained else ''}{start_time}",
    histogram_freq=1,
    write_graph=False,
    profile_batch=0,
)
cnn.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=epochs,
    callbacks=[checkpoint_cb, early_stopping_cb, tensorboard_cb],
)
with file_writer.as_default():
    tf.summary.text(
        "Hyperparameters",
        f"{downscaling=}; "
        f"{encoder_num_filters=}; "
        f"{epochs=}; "
        f"{patience=}; "
        f"{batch_size=}; "
        f"{learning_rate=}; "
        f"{unsupervised_val_perc=}",
        step=0,
    )
cnn = keras.models.load_model(best_checkpoint)

Epoch 1/1000
      2/Unknown - 0s 208ms/step - loss: 6.4859 - accuracy: 0.5000WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0117s vs `on_train_batch_end` time: 0.4037s). Check your callbacks.
     24/Unknown - 9s 395ms/step - loss: 5.2714 - accuracy: 0.5000WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0094s vs `on_test_batch_end` time: 0.0810s). Check your callbacks.

Epoch 00001: val_accuracy improved from -inf to 0.50000, saving model to models/20201023-144533.h5


In [ ]:
cnn = keras.models.load_model("models/20201021-213411.h5")
cnn.evaluate(test_dataset, verbose=0, return_dict=True)

In [ ]:
cnn = keras.models.load_model("models/pretrained-20201021-230015.h5")
cnn.evaluate(test_dataset, verbose=0, return_dict=True)

In [ ]:
x, y = next(iter(test_dataset.skip(6)))
prediction = cnn(x, training=False)
print(f"real: {y.numpy()}, prediction: {prediction.numpy()}")
plot_animated_volume(x[0, :], fps=1)